
## LangGraph + LangChain

This hands-on notebook introduces **LangGraph** and **LangChain (OpenAI)** for agentic AI in Banking/Finance:
- **Intro to LangGraph**
- **LangGraph basic example (typed state)**  
- **LangGraph with GPT (via `langchain-openai`)** *(model name configurable; use `gpt-5` if your account has access)*
- **Use case: LangChain + LangGraph + GPT — banking FAQ summarizer**
- **Multi-Agent Orchestration:** coordinating agents, context passing, decision routing


## LangGraph 

LangGraph is very low-level, and focused entirely on agent orchestration. Before using LangGraph, we recommend you familiarize yourself with some of the components used to build agents, starting with models and tools.


## 0) Setup & Versions
Install/upgrade required packages. If your org pins versions, adapt accordingly.


In [ ]:
%pip install -U langgraph langchain langchain-core langchain-openai python-dotenv

In [1]:
# If needed, uncomment to install/upgrade.
# %pip install -U langgraph langchain langchain-core langchain-openai python-dotenv

import sys, platform, os
print("Python:", sys.version)
print("Platform:", platform.platform())

# Optional: load a .env if present for OPENAI_API_KEY and OPENAI_MODEL
try:
    from dotenv import load_dotenv
    load_dotenv()
    print("Loaded .env")
except Exception as e:
    print("python-dotenv not installed or .env absent (ok).")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-5")  # set to "gpt-5" if your account has access
print("OPENAI_API_KEY set:", bool(OPENAI_API_KEY))
print("OPENAI_MODEL:", OPENAI_MODEL)


Python: 3.11.8 | packaged by conda-forge | (main, Feb 16 2024, 20:49:36) [Clang 16.0.6 ]
Platform: macOS-26.1-arm64-arm-64bit
Loaded .env
OPENAI_API_KEY set: True
OPENAI_MODEL: gpt-5



## 1) LangGraph — What & Why (Quick Intro)

**LangGraph** helps you build **agentic workflows** 
as a graph of nodes with explicit **state**:

- **StateGraph**: define a typed state (keys your nodes read/write).

- **Nodes**: pure functions (sync or async) that transform state.

- **Edges**: connect nodes; use **conditional edges** for **decision routing**.

- **START / END**: special markers to begin and end execution.

This explicit structure is ideal for BFSI where **auditable flows**, **guardrails**, and **determinism** matter.



## 2) LangGraph Basic Example — Typed State, Nodes, Edges

We implement a simple **KYC guard → Retriever → Summarizer** flow.
- If the query contains sensitive hints (e.g., `password`, `otp`, `pin`), we **block**.
- Else we **retrieve** (stub) and **summarize** (stub).


In [94]:
from typing import TypedDict, Literal
from langgraph.graph import StateGraph, START, END

class BFState(TypedDict, total=False):
    query: str
    route: Literal["block", "process"]
    docs: list[str]
    summary: str
    warning: str

RISK_WORDS = {"password", "otp", "pin", "cvv", "secret"}

def kyc_guard(state: BFState) -> BFState:
    q = state["query"].lower()
    if any(w in q for w in RISK_WORDS):
        return {"route": "block", "warning": "🚫 Sensitive/PII-like term detected. Request denied."}
    return {"route": "process"}

def retriever(state: BFState) -> BFState:
    q = state["query"]
    # In production: query vector DB (Chroma/pgvector) or search service
    return {"docs": [f"Policy doc snippet for: {q}",
                     "Interest rate depends on tenure and credit score; see policy 2024-09."]}

def summarizer(state: BFState) -> BFState:
    docs = state.get("docs", [])
    if not docs:
        return {"summary": "No data."}
    return {"summary": "Summary: " + " | ".join(docs)}

graph = StateGraph(BFState)
graph.add_node("kyc_guard", kyc_guard)
graph.add_node("retriever", retriever)
graph.add_node("summarizer", summarizer)

graph.add_edge(START, "kyc_guard")
graph.add_conditional_edges(
    "kyc_guard",
    lambda s: s["route"],
    {"block": END, "process": "retriever"}
)
graph.add_edge("retriever", "summarizer")
graph.add_edge("summarizer", END)

app = graph.compile()

print("SAFE PATH =>", app.invoke({"query": "Latest retail loan policy"}))
print("BLOCK PATH =>", app.invoke({"query": "What is my OTP right now?"}))


SAFE PATH => {'query': 'Latest retail loan policy', 'route': 'process', 'docs': ['Policy doc snippet for: Latest retail loan policy', 'Interest rate depends on tenure and credit score; see policy 2024-09.'], 'summary': 'Summary: Policy doc snippet for: Latest retail loan policy | Interest rate depends on tenure and credit score; see policy 2024-09.'}
BLOCK PATH => {'query': 'What is my OTP right now?', 'route': 'block', 'warning': '🚫 Sensitive/PII-like term detected. Request denied.'}



### 3) LangGraph + GPT (via `langchain-openai`)

We'll add an **LLM summarizer** node powered by OpenAI.  
> Set `OPENAI_API_KEY` and choose a model via `OPENAI_MODEL`. If your workspace has **GPT‑5**, set `OPENAI_MODEL="gpt-5"`.


In [ ]:
USE_LLM = bool(OPENAI_API_KEY)
print("LLM enabled:", USE_LLM)

if USE_LLM:
    from langchain_openai import ChatOpenAI
    from langchain_core.messages import HumanMessage

    llm = ChatOpenAI(model='gpt-4o', temperature=0)

    class LLMState(TypedDict, total=False):
        query: str
        docs: list[str]
        llm_summary: str

    async def llm_summarizer(state: LLMState) -> LLMState:
        docs = state.get("docs") or []
        prompt_text = "Summarize for a banking analyst:\n" + "\n".join(f"- {d}" for d in docs) or "No docs."
        resp = await llm.ainvoke([HumanMessage(content=prompt_text)])
        return {"llm_summary": resp.content}

    from langgraph.graph import StateGraph, START, END

    g_llm = StateGraph(LLMState)
    # reuse retriever from above for demo
    def retriever_llm(state: LLMState) -> LLMState:
        q = state["query"]
        return {"docs": [f"Policy doc snippet for: {q}", "Risk weights revised in RBI circular X."]}

    g_llm.add_node("retriever", retriever_llm)
    g_llm.add_node("llm_summarizer", llm_summarizer)
    g_llm.add_edge(START, "retriever")
    g_llm.add_edge("retriever", "llm_summarizer")
    g_llm.add_edge("llm_summarizer", END)

    app_llm = g_llm.compile()


    import nest_asyncio, asyncio
    nest_asyncio.apply()
    out = await app_llm.ainvoke({"query": "Summarize retail loan policy changes"})
    print(out[""]


LLM enabled: True
{'query': 'Summarize retail loan policy changes', 'docs': ['Policy doc snippet for: Summarize retail loan policy changes', 'Risk weights revised in RBI circular X.'], 'llm_summary': '**Summary for Banking Analyst:**\n\n1. **Retail Loan Policy Changes:**\n   - The policy document outlines recent amendments to the retail loan segment. Key changes include adjustments in eligibility criteria, interest rate structures, and loan-to-value (LTV) ratios. The policy aims to enhance credit accessibility while maintaining prudent risk management. Additionally, there is a focus on digital lending processes to streamline application and approval workflows, improving customer experience and operational efficiency.\n\n2. **RBI Circular X - Revised Risk Weights:**\n   - The Reserve Bank of India (RBI) has issued Circular X, which revises the risk weights applicable to various asset classes. The changes are intended to align with international standards and ensure a more robust risk ma

Script/terminal version (if you move to a .py file)

    Outside Jupyter (plain Python script), keep asyncio.run(...):

In [ ]:
if __name__ == "__main__":
    import asyncio
    out = asyncio.run(app_llm.ainvoke({"query": "Summarize retail loan policy changes"}))
    print(out)


## 4) Use Case: LangChain + LangGraph + GPT — Banking FAQ Summarizer

Flow:
1. Guard → 2. Retrieve FAQs → 3. LLM summarize → 4. Return concise answer

This pattern fits **contact center assistants**, **ops copilots**, or **policy desk bots**.


In [83]:
%%writefile faq_app_langgraph.py

from typing import TypedDict, Literal
from langgraph.graph import StateGraph, START, END
import os

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-5")  # set to "gpt-5" if your account has access
print("OPENAI_API_KEY set:", bool(OPENAI_API_KEY))
print("OPENAI_MODEL:", OPENAI_MODEL)

class FAQState(TypedDict, total=False):
    question: str
    route: Literal["block", "ok"]
    faqs: list[str]
    answer: str
    reason: str

def guard(state: FAQState) -> FAQState:
    q = state["question"].lower()
    if any(x in q for x in ["password", "otp", "cvv", "pin", "secret"]):
        return {"route": "block", "reason": "🚫 Sensitive query blocked."}
    return {"route": "ok"}

def retrieve_faqs(state: FAQState) -> FAQState:
    q = state["question"]
    # Stub: In production, fetch from indexed policies / FAQ DB
    return {"faqs": [f"FAQ: Response policy related to: {q}",
                     "Customers must not share OTP/CVV; see security policy 2025-A."]}

if OPENAI_API_KEY:
    from langchain_openai import ChatOpenAI
    from langchain_core.messages import HumanMessage
    llm2 = ChatOpenAI(model=OPENAI_MODEL, temperature=0)

    async def answer_llm(state: FAQState) -> FAQState:
        faqs = state.get("faqs") or []
        prompt = "Create a concise, compliant answer for a banking customer:\n" + "\n".join(f"- {f}" for f in faqs)
        resp = await llm2.ainvoke([HumanMessage(content=prompt)])
        return {"answer": resp.content}
else:
    def answer_llm(state: FAQState) -> FAQState:
        faqs = state.get("faqs") or []
        return {"answer": "LLM disabled. Heuristic answer: " + " | ".join(faqs)}

faq_graph = StateGraph(FAQState)
faq_graph.add_node("guard", guard)
faq_graph.add_node("retrieve_faqs", retrieve_faqs)
faq_graph.add_node("answer_llm", answer_llm)

faq_graph.add_edge(START, "guard")
faq_graph.add_conditional_edges(
    "guard",
    lambda s: s["route"],
    {"block": END, "ok": "retrieve_faqs"}
)
faq_graph.add_edge("retrieve_faqs", "answer_llm")
faq_graph.add_edge("answer_llm", END)

faq_app = faq_graph.compile()


if __name__ == "__main__":
    import asyncio
    out = asyncio.run(faq_app.ainvoke({"question": "What is the current FD interest rate for 1 year?"}))
    #print("FAQ SAFE PATH =>", out)
    print(out['question'], "=>", out.get('answer'))

    out1 = asyncio.run(faq_app.ainvoke({"question": "Share my OTP please"}))
    print("FAQ BLOCK PATH =>", out1)


Overwriting faq_app_langgraph.py


In [84]:
!python faq_app_langgraph.py

OPENAI_API_KEY set: True
OPENAI_MODEL: gpt-5
What is the current FD interest rate for 1 year? => FAQ: What is the current FD interest rate for 1 year?

FD rates are dynamic and may change without notice. For the latest 1‑year FD rate applicable to your profile:
- Visit our official website (Interest Rates > Fixed Deposits)
- Check our mobile app/NetBanking (Open/Book FD to view live rates for your amount and tenor)
- Visit your branch or call our official helpline

Note: Rates can vary by customer category (e.g., senior citizen), deposit amount, tenor, and payout option.

Security reminder (Policy 2025-A): Never share your OTP, CVV, card/PIN, or passwords. We will never ask for these over calls, emails, chat, or links. If unsure, contact us via official channels only.
FAQ BLOCK PATH => {'question': 'Share my OTP please', 'route': 'block', 'reason': '🚫 Sensitive query blocked.'}



## 5) Multi‑Agent Orchestration — Coordinating Agents, Context Passing, Decision Routing

We build a small **router** that sends queries to one of two agents:
- **Calc Agent**: safely evaluates basic arithmetic (no `eval`).
- **Policy Agent**: returns policy-search stubs (replace with vector DB later).

**Context Passing**: both agents read from the same typed state; router sets a `route` key.


In [92]:

import ast, operator as op
from typing import TypedDict, Literal
from langgraph.graph import StateGraph, START, END

OPS = {ast.Add: op.add, ast.Sub: op.sub, ast.Mult: op.mul, ast.Div: op.truediv}

class RouteState(TypedDict, total=False):
    query: str
    route: Literal["calc_agent", "policy_agent"]
    result: str

def router(state: RouteState) -> RouteState:
    q = state["query"].lower()
    if any(tok in q for tok in ["+", "-", "*", "/", "sum", "calculate"]):
        return {"route": "calc_agent"}
    return {"route": "policy_agent"}

def calc_agent(state: RouteState) -> RouteState:
    text = state["query"].lower().replace("sum", "").replace("calculate", "").strip()
    try:
        node = ast.parse(text, mode="eval").body
        def _eval(n):
            if isinstance(n, ast.Num):  # type: ignore[attr-defined]
                return float(n.n)
            if isinstance(n, ast.BinOp) and type(n.op) in OPS:
                return OPS[type(n.op)](_eval(n.left), _eval(n.right))
            raise ValueError("Only + - * / supported.")
        val = _eval(node)
        return {"result": f"{val:g}"}
    except Exception:
        return {"result": "Failed to parse expression."}

def policy_agent(state: RouteState) -> RouteState:
    q = state["query"]
    return {"result": f"🔎 (Stub) Policy search result for: {q}"}

rg = StateGraph(RouteState)
rg.add_node("router", router)
rg.add_node("calc_agent", calc_agent)
rg.add_node("policy_agent", policy_agent)

rg.add_edge(START, "router")
rg.add_conditional_edges(
    "router",
    lambda s: s["route"],
    {"calc_agent": "calc_agent", "policy_agent": "policy_agent"}
)
rg.add_edge("calc_agent", END)
rg.add_edge("policy_agent", END)

route_app = rg.compile()

import asyncio

out= asyncio.run(route_app.ainvoke({"query": "sum 18 + 24"}))
print(out["query"], "=>", out["result"])

out1 = asyncio.run(route_app.ainvoke({"query": "policy for retail loans under 25L?"}))
#print(out1)
print(out1["query"], "=>", out1["result"])

sum 18 + 24 => 42
policy for retail loans under 25L? => 🔎 (Stub) Policy search result for: policy for retail loans under 25L?
